In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr
from scipy import stats
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import seaborn as sns
from datetime import datetime


/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Install File


In [2]:
nameCSV = 'innis_1.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]
back =data.values[:,5]
ws = data.values[:,6]
len(lat)

2662

## Calculate mean value of each satellite track 

In [3]:
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            
            
print 'Number of Satellite Tracks:', len(tt[days])

Number of Satellite Tracks: 2394


## Make ndarray(alltime) and list(days2) that has dates

In [4]:
alltime = num2date(tt[:],u'days since 1985-01-01 00:00:00 UTC')

In [5]:
netCDF4.num2date(tt[days],u'days since 1985-01-01 00:00:00 UTC')

array([datetime.datetime(1985, 3, 31, 8, 13, 46, 832721),
       datetime.datetime(1985, 4, 5, 21, 26, 15, 503909),
       datetime.datetime(1985, 4, 8, 21, 35, 6, 187500), ...,
       datetime.datetime(2019, 2, 9, 1, 11, 43, 638834),
       datetime.datetime(2019, 2, 10, 13, 55, 59, 960156),
       datetime.datetime(2019, 2, 18, 23, 10, 14, 196875)], dtype=object)

In [6]:
for k in range(len(days)):
    print wh[days[k]], alltime[k]

2.1770000457763667 1985-03-31 08:13:46.832721
0.29100000858306885 1985-04-05 21:26:15.503909
2.1640000343322754 1985-04-08 21:35:06.187500
1.579800009727478 1985-04-23 09:53:29.146876
2.3106002807617188 1985-05-16 11:33:14.298438
0.4826667308807373 1985-07-30 16:49:35.815105
1.2717777490615845 1985-08-22 18:29:05.607639
0.4731818437576294 1985-09-14 20:08:44.546877
0.4109000265598297 1985-09-23 09:29:54.689844
2.001000165939331 1985-10-07 21:48:02.343750
1.241000056266785 1985-10-30 23:27:24.464844
1.731545329093933 1985-11-02 23:36:09.269886
1.1175554990768433 1985-11-08 12:48:30.101562
0.5516666769981384 1985-11-26 01:15:21.096897
0.7686000466346741 1986-01-11 04:33:36.420703
0.6541999578475952 1986-01-19 17:54:59.206250
1.075285792350769 1986-02-03 06:12:35.508371
1.744444489479065 1986-03-01 08:00:17.441840
2.001062393188477 1986-03-24 09:39:13.101562
2.3270528316497803 1986-04-16 11:18:06.899671
0.5181428790092468 1986-05-09 12:56:55.488838
2.768230676651001 1986-06-01 14:35:32.74

0.5535000562667847 2000-07-15 17:58:14.170230
1.4794737100601196 2000-07-16 06:04:17.524690
1.3353159427642822 2000-07-17 06:42:31.736636
1.1824705600738523 2000-08-01 12:49:51.849725
1.2985000610351562 2000-08-04 00:30:54.717865
1.843666672706604 2000-08-04 13:55:17.974506
0.7413684129714966 2000-08-06 02:39:36.410876
0.6968947052955627 2000-08-14 11:53:49.861431
1.2854211330413818 2000-08-16 00:38:08.552169
2.4672632217407227 2000-08-20 05:31:28.051975
1.2233500480651855 2000-08-23 11:59:35.539058
1.1800527572631836 2000-08-24 09:52:21.877056
1.9353530406951904 2000-08-25 22:36:41.455080
1.3794287443161009 2000-09-03 07:50:53.177632
1.549950122833252 2000-09-04 20:35:10.596217
1.1969000101089478 2000-09-05 12:49:57.023437
0.7524737119674683 2000-09-08 00:30:57.267166
2.8682000637054443 2000-09-13 05:49:23.507812
1.2747143507003784 2000-09-14 18:33:41.799342
1.1893999576568604 2000-09-23 03:47:54.289258
1.474000096321106 2000-09-24 05:25:59.548178
1.4463158845901491 2000-09-24 16:32:1

1.2915000915527344 2004-04-10 23:58:00.826041
0.8985334634780884 2004-04-11 12:22:41.135655
1.0110666751861572 2004-04-16 21:06:55.238839
0.659500002861023 2004-04-27 12:19:51.738021
0.5083333849906921 2004-04-30 00:00:53.465800
0.6062727570533752 2004-05-01 11:52:14.071615
1.297714352607727 2004-05-15 23:58:03.069792
1.1370714902877808 2004-05-16 12:22:44.716407
0.6586363315582275 2004-05-18 13:05:02.855470
1.3291666507720947 2004-06-01 12:19:56.850587
0.4312000274658203 2004-06-04 00:00:58.086499
0.4456250071525574 2004-06-04 14:17:52.525003
0.456428587436676 2004-06-19 23:58:05.827083
0.5467334389686584 2004-06-20 12:22:46.694662
1.5381934642791748 2004-06-21 15:30:43.692707
0.3350000083446503 2004-07-06 12:19:55.998958
1.4542001485824585 2004-07-08 16:43:32.640623
1.4137272834777832 2004-07-09 00:00:58.280800
0.6163333654403687 2004-07-24 23:58:07.422917
0.9611999392509459 2004-07-25 12:22:48.289714
1.1972668170928955 2004-07-25 17:56:22.209375
1.0381667613983154 2004-08-10 12:19:5

0.5500000224393957 2010-02-13 23:57:45.119791
1.9443259861158289 2010-02-14 12:22:25.431641
1.6845715386526925 2010-02-15 12:02:26.318750
1.301846265792847 2010-02-23 21:16:39.860939
0.5686249732971191 2010-02-25 10:00:57.724610
0.2802143096923828 2010-03-02 12:19:35.990625
1.1123499870300293 2010-03-05 00:00:37.941300
1.6573809669131323 2010-03-06 07:29:55.249999
3.431450271606445 2010-03-07 07:59:33.872656
3.3426190330868675 2010-03-15 17:13:42.119531
0.5880000591278076 2010-03-18 08:12:52.943614
1.5305999755859374 2010-03-20 23:57:46.138542
0.2953529638402602 2010-03-21 12:22:27.563210
2.5495335896809896 2010-03-25 17:14:41.837457
2.4538825539981617 2010-03-27 03:56:31.917763
1.738196849822998 2010-04-04 16:31:06.819203
1.3193001747131348 2010-04-06 01:55:02.931641
1.5746666590372722 2010-04-06 12:19:36.255730
2.285166634453668 2010-04-09 00:00:37.589380
2.5438889397515188 2010-04-14 11:09:14.937109
2.526722378200955 2010-04-15 23:53:32.900782
2.2221177045036766 2010-04-24 09:07:46.

2.2447223663330083 2013-07-07 10:13:46.731640
0.9892001152038574 2013-07-08 22:58:04.961718
1.0081667370266385 2013-07-17 08:12:17.816406
1.0094075379548249 2013-07-18 20:56:35.989843
1.8874617356520436 2013-07-21 08:43:54.989430
1.4859001159667968 2013-07-26 20:21:52.792279
1.064900016784668 2013-07-27 06:10:48.635742
1.1252500534057617 2013-07-28 18:55:07.006250
1.858000138226677 2013-08-06 04:09:19.397070
2.424949836730957 2013-08-07 16:53:37.719532
2.4750000448787914 2013-08-16 02:07:50.916308
0.6268500328063965 2013-08-17 07:21:20.884765
0.5051428590502057 2013-08-17 14:52:09.250000
1.278500123457475 2013-08-19 07:18:49.872244
0.5919091484763406 2013-08-24 18:56:46.899816
0.3555263218126799 2013-08-26 00:06:23.354278
0.9883999824523926 2013-08-27 12:50:41.561719
2.7720000743865967 2013-09-04 22:04:55.514844
3.1444499969482416 2013-09-14 20:03:27.195313
2.0638097127278647 2013-09-16 08:47:45.889063
2.9463752110799155 2013-09-16 16:16:31.967222
0.7566500663757324 2013-09-20 17:34:17

1.2603529691696167 2017-03-26 20:53:23.812499
0.9741001129150392 2017-03-28 09:37:39.003126
0.6843333017258417 2017-04-04 00:00:09
0.4541052768104954 2017-04-05 18:51:52.337500
0.6448824265423942 2017-04-07 07:36:10.066015
0.4621666669845581 2017-04-10 13:20:16.638020
0.4196875095367432 2017-04-12 12:20:17.000001
0.829800033569336 2017-04-15 16:50:23.343749
3.446500015258789 2017-04-16 00:58:17.085018
1.3105002403259278 2017-04-17 05:34:41.415821
0.7486000061035156 2017-04-25 14:48:55.657030
1.9384117126464844 2017-04-27 03:33:14.006882
0.7117619287400019 2017-05-01 00:00:11
0.9311200714111328 2017-05-05 12:47:27.753125
0.6274667104085286 2017-05-07 01:31:45.955420
2.984353009392233 2017-05-09 11:55:13.369026
2.7652354520909928 2017-05-10 12:59:12.666667
2.548400115966797 2017-05-14 23:33:09.051470
2.2562855311802457 2017-05-15 14:06:22.528054
1.0772500038146973 2017-05-16 23:30:19.381579
1.480200004577637 2017-05-25 08:44:31.914063
1.5783530473709106 2017-05-26 21:28:50.544531
1.68495

In [7]:
days2 = [] #daystart
for k in range(len(alltime)):
       
    t1 = alltime[k]
    if k == 0: #get day 0. Then else: every day after that
        days2.append(alltime[k])
        dd =alltime[k]
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days2.append(alltime[k])
            it += 1
            dd = alltime[k]
            
            
print 'Number of Satellite Tracks:', len(days2)

Number of Satellite Tracks: 2394


In [8]:
days2

[datetime.datetime(1985, 3, 31, 8, 13, 46, 832721),
 datetime.datetime(1985, 4, 5, 21, 26, 15, 503909),
 datetime.datetime(1985, 4, 8, 21, 35, 6, 187500),
 datetime.datetime(1985, 4, 23, 9, 53, 29, 146876),
 datetime.datetime(1985, 5, 16, 11, 33, 14, 298438),
 datetime.datetime(1985, 7, 30, 16, 49, 35, 815105),
 datetime.datetime(1985, 8, 22, 18, 29, 5, 607639),
 datetime.datetime(1985, 9, 14, 20, 8, 44, 546877),
 datetime.datetime(1985, 9, 23, 9, 29, 54, 689844),
 datetime.datetime(1985, 10, 7, 21, 48, 2, 343750),
 datetime.datetime(1985, 10, 30, 23, 27, 24, 464844),
 datetime.datetime(1985, 11, 2, 23, 36, 9, 269886),
 datetime.datetime(1985, 11, 8, 12, 48, 30, 101562),
 datetime.datetime(1985, 11, 26, 1, 15, 21, 96897),
 datetime.datetime(1986, 1, 11, 4, 33, 36, 420703),
 datetime.datetime(1986, 1, 19, 17, 54, 59, 206250),
 datetime.datetime(1986, 2, 3, 6, 12, 35, 508371),
 datetime.datetime(1986, 3, 1, 8, 0, 17, 441840),
 datetime.datetime(1986, 3, 24, 9, 39, 13, 101562),
 datetime.

## WAVE PERIOD EQN

$ \epsilon = 3.25 \left( H_s^2 g^2/U^4 \right)^{0.31}$



In [9]:
def waveage(H, U, grav=9.80665):
    '''
    The pseudo wave age can be expressed in terms of significant wave height and surface wind speed.
    '''

    grav2 = grav**2
    hs2 = np.square(H) 
    u4 = np.power(U,4)
    tmp = np.divide(hs2*grav2,u4)
    eps = 3.25*np.power(tmp,0.31)
    
    return eps


def waveperiod(H, U, grav=9.80665):
    '''
    Wave age from GA-2, Govindan et al.
    '''

    eps = waveage(H, U, grav=9.80665)
    period = (((eps-(5.78))/(eps+(U/(H*((U/H)+H)))))+(H+(5.70)))
    
    return period

In [10]:
T = waveperiod(wh,ws)
print T
print len(T),len(wh)

[7.21842117 3.84835449 6.06049484 ... 5.92469211 6.59358157 5.14627429]
2662 2662


In [11]:
dayswh = wh[days] #wave height for each satellite pass
daysTz = T[days]

print len(dayswh)
print len(daysTz)

2394
2394


## Calculate Wave Power



### Total Wave Energy
$E = \frac{1}{8} \left(pgH_s^2\right) $


In [12]:
def totalwaveenergy(H):
    
    '''
    The total wave energy can be calculated using Hs, gravity and water density
    '''
    
    x = 1./8.
    p = 1025. #sea water density, kg/m^3
    g = 9.80665
    h2 = np.square(H) 
    
    pgh = p*g*h2
    
    e = x*pgh
    
    return e

In [13]:
we = totalwaveenergy(wh)

In [14]:
we

array([5954.85828247,  106.39973776, 5883.95145043, ..., 6377.57462625,
       1880.42510514, 1720.57967131])

### Wave energy speed / Wave Velocity
$Cg = \frac{gTz}{2\pi}n$


In [17]:
import math
def wavespeed(T):
    '''
    Wave group speed
    '''
    
    g =9.80665
    ts = T #wave period
    pi = 2*math.pi
    n=0.5
    
    top = g*ts
    
    Cg = n *(np.divide(top,pi))
    
    return Cg
    

In [18]:
Cg = wavespeed(T)
Cg

array([5.63317223, 3.00321124, 4.72953994, ..., 4.62356107, 5.14555464,
       4.01609282])

### Wave Energy Flux
$P = ECg$

In [19]:
def waveenergyflux(H,T):
    '''
    The rate at which energy is carried by waves, determined using total wave energy and wave energy speed
    '''
    
    e = totalwaveenergy(H)
    Cg = wavespeed(T)
    y = 0.001
    P = e * Cg*y
    
    return P
    
    

In [20]:
power1 = waveenergyflux(wh,T)
power1

array([33.5447423 ,  0.31954089, 27.82838339, ..., 29.48710573,
        9.67583013,  6.91000766])

In [21]:
pp = power1[days]

In [22]:
print pp.shape, power1.shape, 'therefore use pp in df'

(2394,) (2662,) therefore use pp in df


In [23]:
print dayswh.shape
print daysTz.shape

(2394,)
(2394,)


In [24]:
df = pd.DataFrame(data={"date": days2, "wh":dayswh, "period":daysTz, "power":pp})
df.to_csv("innis_timeseries.csv", sep=',',index=False)
df

date    period      power        wh
0    1985-03-31 08:13:46.832721  7.218421  33.544742  2.177000
1    1985-04-05 21:26:15.503909  3.848354   0.319541  0.291000
2    1985-04-08 21:35:06.187500  6.060495  27.828383  2.164000
3    1985-04-23 09:53:29.146876  4.812704  11.777651  1.579800
4    1985-05-16 11:33:14.298438  5.343270  27.971910  2.310600
5    1985-07-30 16:49:35.815105  4.298772   0.981984  0.482667
6    1985-08-22 18:29:05.607639  6.605341  10.475701  1.271778
7    1985-09-14 20:08:44.546877  3.655896   0.802630  0.473182
8    1985-09-23 09:29:54.689844  3.871910   0.641007  0.410900
9    1985-10-07 21:48:02.343750  5.364824  21.062751  2.001000
10   1985-10-30 23:27:24.464844  4.480112   6.765468  1.241000
11   1985-11-02 23:36:09.269886  4.964859  14.596208  1.731545
12   1985-11-08 12:48:30.101562  5.155676   6.313777  1.117555
13   1985-11-26 01:15:21.096897  3.529332   1.053201  0.551667
14   1986-01-11 04:33:36.420703  6.388519   3.700551  0.768600
15   1986-01-19 17:54:59.206250  4.579486   1.921777  0.654200
16   1986-02-03 06:12:35.508371  4.946787   5.608367  1.075286
17   1986-03-01 08:00:17.441840  5.883476  17.555518  1.744444
18   1986-03-24 09:39:13.101562  5.461074  21.441972  2.001062
19   1986-04-16 11:18:06.899671  5.962222  31.658183  2.327053
20   1986-05-09 12:56:55.488838  3.826248   1.007251  0.518143
21   1986-06-01 14:35:32.742789  5.740774  43.136051  2.768231
22   1986-06-24 16:14:01.945314  5.193907  12.184279  1.546750
23   1986-07-17 17:52:39.277575  5.404645  10.046763  1.376882
24   1986-09-01 21:09:37.889649  5.864396  35.232924  2.475312
25   1986-09-10 10:30:46.465278  5.581465  15.110273  1.661611
26   1986-09-24 22:48:05.623699  4.019368   2.148464  0.738333
27   1986-11-09 14:50:45.132812  4.213162   2.035860  0.702000
28   1986-11-26 16:03:29.375003  6.019693  12.616372  1.462000
29   1987-02-02 20:54:26.062498  6.258021  22.035394  1.895000
...                         ...       ...        ...       ...
2364 2018-11-21 19:08:25.734747  4.673469   2.146051  0.684333
2365 2018-11-23 06:07:46.802631  4.665537   0.943366  0.454105
2366 2018-11-27 07:49:35.633732  4.278931   1.744864  0.644882
2367 2018-12-01 03:38:59.973047  4.129310   0.864849  0.462167
2368 2018-12-02 19:27:36.187500  4.718134   0.814869  0.419688
2369 2018-12-03 04:06:16.772656  5.838010   3.941641  0.829800
2370 2018-12-07 13:14:13.011719  6.394466  74.477695  3.446500
2371 2018-12-11 13:20:30.168750  5.328405   8.972982  1.310500
2372 2018-12-13 02:04:47.866016  4.802386   2.638895  0.748600
2373 2018-12-16 00:00:11.000000  5.179700  19.083675  1.938412
2374 2018-12-18 00:09:10.915397  5.459009   2.711744  0.711762
2375 2018-12-21 18:39:51.439257  4.671923   3.971663  0.931120
2376 2018-12-23 00:03:22.027146  4.294693   1.657978  0.627467
2377 2018-12-24 06:26:58.213925  5.879304  51.344237  2.984353
2378 2018-12-29 18:05:01.030331  5.694865  42.698538  2.765235
2379 2018-12-31 09:17:33.707032  5.930561  37.765601  2.548400
2380 2019-01-01 22:01:51.890625  5.495538  27.432394  2.256286
2381 2019-01-10 06:14:39.021484  4.814379   5.478210  1.077250
2382 2019-01-11 20:00:23.753125  4.691990  10.080063  1.480200
2383 2019-01-12 00:00:14.000000  5.088167  12.428966  1.578353
2384 2019-01-20 05:14:38.116211  4.980003  13.863377  1.684950
2385 2019-01-21 17:58:56.040625  4.949844  10.403953  1.464100
2386 2019-01-22 05:01:58.905331  4.982491   9.756741  1.413177
2387 2019-01-27 16:39:57.302390  5.147659  42.268732  2.893824
2388 2019-01-30 13:42:56.420689  5.472712  23.195669  2.079072
2389 2019-02-02 05:41:24.830017  4.561774  17.947968  2.003125
2390 2019-02-08 00:00:17.000000  7.015937  36.030109  2.288533
2391 2019-02-09 01:11:43.638834  5.924692  29.487106  2.252945
2392 2019-02-10 13:55:59.960156  6.593582   9.675830  1.223350
2393 2019-02-18 23:10:14.196875  5.146274   6.910008  1.170200

[2394 rows x 4 columns]

In [42]:
df99 = pd.DataFrame(data={"date": days2, "wh":dayswh})
df99.to_csv("innis_wh_long.csv", sep=',',index=False)
df99

date        wh
0    1985-03-31 08:13:46.832721  2.177000
1    1985-04-05 21:26:15.503909  0.291000
2    1985-04-08 21:35:06.187500  2.164000
3    1985-04-23 09:53:29.146876  1.579800
4    1985-05-16 11:33:14.298438  2.310600
5    1985-07-30 16:49:35.815105  0.482667
6    1985-08-22 18:29:05.607639  1.271778
7    1985-09-14 20:08:44.546877  0.473182
8    1985-09-23 09:29:54.689844  0.410900
9    1985-10-07 21:48:02.343750  2.001000
10   1985-10-30 23:27:24.464844  1.241000
11   1985-11-02 23:36:09.269886  1.731545
12   1985-11-08 12:48:30.101562  1.117555
13   1985-11-26 01:15:21.096897  0.551667
14   1986-01-11 04:33:36.420703  0.768600
15   1986-01-19 17:54:59.206250  0.654200
16   1986-02-03 06:12:35.508371  1.075286
17   1986-03-01 08:00:17.441840  1.744444
18   1986-03-24 09:39:13.101562  2.001062
19   1986-04-16 11:18:06.899671  2.327053
20   1986-05-09 12:56:55.488838  0.518143
21   1986-06-01 14:35:32.742789  2.768231
22   1986-06-24 16:14:01.945314  1.546750
23   1986-07-17 17:52:39.277575  1.376882
24   1986-09-01 21:09:37.889649  2.475312
25   1986-09-10 10:30:46.465278  1.661611
26   1986-09-24 22:48:05.623699  0.738333
27   1986-11-09 14:50:45.132812  0.702000
28   1986-11-26 16:03:29.375003  1.462000
29   1987-02-02 20:54:26.062498  1.895000
...                         ...       ...
2364 2018-11-21 19:08:25.734747  0.684333
2365 2018-11-23 06:07:46.802631  0.454105
2366 2018-11-27 07:49:35.633732  0.644882
2367 2018-12-01 03:38:59.973047  0.462167
2368 2018-12-02 19:27:36.187500  0.419688
2369 2018-12-03 04:06:16.772656  0.829800
2370 2018-12-07 13:14:13.011719  3.446500
2371 2018-12-11 13:20:30.168750  1.310500
2372 2018-12-13 02:04:47.866016  0.748600
2373 2018-12-16 00:00:11.000000  1.938412
2374 2018-12-18 00:09:10.915397  0.711762
2375 2018-12-21 18:39:51.439257  0.931120
2376 2018-12-23 00:03:22.027146  0.627467
2377 2018-12-24 06:26:58.213925  2.984353
2378 2018-12-29 18:05:01.030331  2.765235
2379 2018-12-31 09:17:33.707032  2.548400
2380 2019-01-01 22:01:51.890625  2.256286
2381 2019-01-10 06:14:39.021484  1.077250
2382 2019-01-11 20:00:23.753125  1.480200
2383 2019-01-12 00:00:14.000000  1.578353
2384 2019-01-20 05:14:38.116211  1.684950
2385 2019-01-21 17:58:56.040625  1.464100
2386 2019-01-22 05:01:58.905331  1.413177
2387 2019-01-27 16:39:57.302390  2.893824
2388 2019-01-30 13:42:56.420689  2.079072
2389 2019-02-02 05:41:24.830017  2.003125
2390 2019-02-08 00:00:17.000000  2.288533
2391 2019-02-09 01:11:43.638834  2.252945
2392 2019-02-10 13:55:59.960156  1.223350
2393 2019-02-18 23:10:14.196875  1.170200

[2394 rows x 2 columns]

In [25]:
df[400:500]

date    period      power        wh
400 1997-04-13 05:20:30.439829  5.320210  19.116470  1.914286
401 1997-04-14 09:39:34.628906  5.832970  27.781372  2.203938
402 1997-04-22 18:53:49.785157  5.822603  39.358608  2.625600
403 1997-04-24 07:38:07.876645  5.722450  30.139514  2.317632
404 1997-04-29 12:49:46.913604  5.819298  38.151707  2.585765
405 1997-05-02 00:30:48.761624  5.660149  20.054265  1.900889
406 1997-05-04 05:36:39.703125  6.011254  30.163126  2.262158
407 1997-05-12 14:50:56.120192  4.710454   5.151330  1.056077
408 1997-05-14 03:35:11.515419  4.938191   7.725890  1.263158
409 1997-05-18 05:25:50.917465  4.918468   4.951243  1.013233
410 1997-05-22 12:49:25.296484  5.272967   8.278312  1.265350
411 1997-05-24 01:33:43.654348  5.701453   2.545981  0.674842
412 1997-06-01 10:47:57.174753  5.085017  10.269690  1.435158
413 1997-06-02 23:32:16.178820  5.217826  13.389454  1.617722
414 1997-06-06 00:30:51.673441  5.397667  12.385705  1.529765
415 1997-06-21 06:45:00.559245  6.202894  56.268736  3.041611
416 1997-06-22 05:25:52.679915  5.649464  29.523652  2.308600
417 1997-07-01 04:43:30.593339  5.529089   9.671468  1.335632
418 1997-07-02 17:27:48.575657  5.155504  11.823089  1.529316
419 1997-07-08 12:49:50.826286  5.606779   6.369249  1.076353
420 1997-07-11 00:30:52.801925  5.846565  28.296467  2.221688
421 1997-07-12 15:26:19.517269  5.708047  30.934194  2.350948
422 1997-07-21 00:40:33.368218  6.557835  80.025477  3.527778
423 1997-07-22 13:24:48.865885  6.204568  65.296044  3.276084
424 1997-07-27 12:52:41.210938  5.156635   8.295012  1.280833
425 1997-07-30 22:39:04.794408  5.714447  29.101015  2.278947
426 1997-08-01 11:23:22.714844  5.313000  23.587418  2.127833
427 1997-08-12 12:49:50.550782  5.426589   1.915557  0.600000
428 1997-08-15 00:30:52.030733  5.369911  21.215561  2.007294
429 1997-08-19 18:36:06.832032  5.753226  25.787996  2.138062
..                         ...       ...        ...       ...
470 1998-02-05 20:55:21.287007  4.532595   2.941029  0.813474
471 1998-02-06 00:30:47.916662  4.575116   2.778543  0.787000
472 1998-02-14 06:09:39.328906  5.959432   4.054715  0.833000
473 1998-02-15 18:53:53.689453  6.130778   4.715656  0.885688
474 1998-02-22 12:52:33.597655  4.601221   5.336470  1.087572
475 1998-02-24 04:08:07.568360  5.069740  13.535828  1.650125
476 1998-02-25 16:52:24.929276  5.216087  11.027584  1.468369
477 1998-03-06 02:06:38.937808  5.787784  20.754907  1.912368
478 1998-03-07 14:50:56.912830  5.187669   4.040635  0.891263
479 1998-03-10 12:49:46.276195  5.194748  21.589536  2.058765
480 1998-03-13 00:30:48.288316  5.305133  18.275298  1.874353
481 1998-03-16 00:05:10.938367  5.122621   9.030524  1.340842
482 1998-03-17 12:49:28.731250  5.089867  13.414301  1.639450
483 1998-03-25 22:03:42.315104  6.618413  13.828010  1.459722
484 1998-03-27 10:48:00.104296  5.906817   9.200286  1.260350
485 1998-03-29 05:50:37.609131  4.538605   2.063667  0.680967
486 1998-04-04 20:02:13.805921  6.107863  48.656478  2.850316
487 1998-04-06 08:46:31.867599  5.728382  34.815089  2.489632
488 1998-04-14 12:49:46.375000  4.768416   5.748632  1.108824
489 1998-04-16 06:45:02.923931  5.306508  14.576343  1.673737
490 1998-04-17 00:30:48.400247  5.147561   7.966170  1.256294
491 1998-04-24 15:59:16.406250  4.632958   2.571850  0.752421
492 1998-04-26 04:43:36.809895  5.456483   3.933804  0.857467
493 1998-05-03 05:50:36.974991  5.127584  16.267156  1.798733
494 1998-05-04 13:57:49.045833  5.500000  14.968494  1.666000
495 1998-05-06 02:42:06.070508  4.595206   2.930752  0.806500
496 1998-05-14 11:56:20.233073  5.697374  28.008605  2.239111
497 1998-05-16 00:40:38.242265  5.375462  18.615756  1.879316
498 1998-05-19 12:49:44.584099  4.404044   2.104623  0.698118
499 1998-05-22 00:30:45.570184  4.569316   5.258435  1.083353

[100 rows x 4 columns]

In [26]:
print len(df.wh)
print len(df.date)
print len(df.power)

2394
2394
2394


## Get rolling average wh based on 30 Days

In [27]:
yrolling = df.rolling('30D', on = 'date', min_periods = 1).mean()

In [28]:
yrolling

date    period      power        wh
0    1985-03-31 08:13:46.832721  7.218421  33.544742  2.177000
1    1985-04-05 21:26:15.503909  5.533388  16.932142  1.234000
2    1985-04-08 21:35:06.187500  5.709090  20.564222  1.544000
3    1985-04-23 09:53:29.146876  5.484994  18.367580  1.552950
4    1985-05-16 11:33:14.298438  5.077987  19.874781  1.945200
5    1985-07-30 16:49:35.815105  4.298772   0.981984  0.482667
6    1985-08-22 18:29:05.607639  5.452057   5.728842  0.877222
7    1985-09-14 20:08:44.546877  5.130619   5.639165  0.872480
8    1985-09-23 09:29:54.689844  3.763903   0.721819  0.442041
9    1985-10-07 21:48:02.343750  4.297543   7.502129  0.961694
10   1985-10-30 23:27:24.464844  4.922468  13.914109  1.621000
11   1985-11-02 23:36:09.269886  4.936598  14.141475  1.657849
12   1985-11-08 12:48:30.101562  4.866883   9.225151  1.363367
13   1985-11-26 01:15:21.096897  4.532495   7.182163  1.160442
14   1986-01-11 04:33:36.420703  6.388519   3.700551  0.768600
15   1986-01-19 17:54:59.206250  5.484003   2.811164  0.711400
16   1986-02-03 06:12:35.508371  5.304931   3.743565  0.832695
17   1986-03-01 08:00:17.441840  5.415131  11.581942  1.409865
18   1986-03-24 09:39:13.101562  5.672275  19.498745  1.872753
19   1986-04-16 11:18:06.899671  5.711648  26.550078  2.164058
20   1986-05-09 12:56:55.488838  4.894235  16.332717  1.422598
21   1986-06-01 14:35:32.742789  4.783511  22.071651  1.643187
22   1986-06-24 16:14:01.945314  5.467341  27.660165  2.157490
23   1986-07-17 17:52:39.277575  5.299276  11.115521  1.461816
24   1986-09-01 21:09:37.889649  5.864396  35.232924  2.475312
25   1986-09-10 10:30:46.465278  5.722930  25.171598  2.068462
26   1986-09-24 22:48:05.623699  5.155076  17.497220  1.625086
27   1986-11-09 14:50:45.132812  4.213162   2.035860  0.702000
28   1986-11-26 16:03:29.375003  5.116427   7.326116  1.082000
29   1987-02-02 20:54:26.062498  6.258021  22.035394  1.895000
...                         ...       ...        ...       ...
2364 2018-11-21 19:08:25.734747  4.759233   6.330073  1.069283
2365 2018-11-23 06:07:46.802631  4.741180   6.538602  1.078335
2366 2018-11-27 07:49:35.633732  4.740073   6.466835  1.066500
2367 2018-12-01 03:38:59.973047  4.788256   6.764542  1.084927
2368 2018-12-02 19:27:36.187500  4.821506   6.314989  1.017467
2369 2018-12-03 04:06:16.772656  4.906215   6.117210  1.001828
2370 2018-12-07 13:14:13.011719  4.956869  10.986951  1.114929
2371 2018-12-11 13:20:30.168750  4.987830  10.819120  1.131226
2372 2018-12-13 02:04:47.866016  4.965406   9.824893  1.050969
2373 2018-12-16 00:00:11.000000  4.988010  10.655145  1.097787
2374 2018-12-18 00:09:10.915397  5.024241  10.044114  1.068092
2375 2018-12-21 18:39:51.439257  5.011607  10.192691  1.048489
2376 2018-12-23 00:03:22.027146  4.980042  10.152018  1.043750
2377 2018-12-24 06:26:58.213925  5.081189  14.352091  1.254604
2378 2018-12-29 18:05:01.030331  5.199184  17.764897  1.431300
2379 2018-12-31 09:17:33.707032  5.349288  20.839960  1.605153
2380 2019-01-01 22:01:51.890625  5.414072  23.058087  1.758203
2381 2019-01-10 06:14:39.021484  5.231887  18.523265  1.627217
2382 2019-01-11 20:00:23.753125  5.174032  18.623909  1.642644
2383 2019-01-12 00:00:14.000000  5.166876  18.107664  1.637286
2384 2019-01-20 05:14:38.116211  5.154142  20.672103  1.793361
2385 2019-01-21 17:58:56.040625  5.181934  21.315332  1.846659
2386 2019-01-22 05:01:58.905331  5.250714  22.125208  1.925230
2387 2019-01-27 16:39:57.302390  5.177550  21.217657  1.916177
2388 2019-01-30 13:42:56.420689  5.069198  17.212012  1.769690
2389 2019-02-02 05:41:24.830017  4.965446  16.158187  1.741561
2390 2019-02-08 00:00:17.000000  5.170496  18.145379  1.796258
2391 2019-02-09 01:11:43.638834  5.239059  19.176445  1.837775
2392 2019-02-10 13:55:59.960156  5.400805  19.558047  1.851057
2393 2019-02-18 23:10:14.196875  5.477497  19.953949  1.847328

[2394 rows x 4 columns]

In [29]:
wh_rolling=yrolling['wh']

period_rolling=yrolling['period']

power_rolling = yrolling['power']

## Incorporate rolling average wh into dataframe

In [30]:
df = pd.DataFrame(data={"date": days2, "wh":dayswh, "wh_rolling":wh_rolling, "period":daysTz, "period_rolling":period_rolling, "power":pp, "power_rolling":power_rolling })
df.to_csv("innis_timeseries.csv", sep=',',index=False)
df

date    period  period_rolling      power  \
0    1985-03-31 08:13:46.832721  7.218421        7.218421  33.544742   
1    1985-04-05 21:26:15.503909  3.848354        5.533388   0.319541   
2    1985-04-08 21:35:06.187500  6.060495        5.709090  27.828383   
3    1985-04-23 09:53:29.146876  4.812704        5.484994  11.777651   
4    1985-05-16 11:33:14.298438  5.343270        5.077987  27.971910   
5    1985-07-30 16:49:35.815105  4.298772        4.298772   0.981984   
6    1985-08-22 18:29:05.607639  6.605341        5.452057  10.475701   
7    1985-09-14 20:08:44.546877  3.655896        5.130619   0.802630   
8    1985-09-23 09:29:54.689844  3.871910        3.763903   0.641007   
9    1985-10-07 21:48:02.343750  5.364824        4.297543  21.062751   
10   1985-10-30 23:27:24.464844  4.480112        4.922468   6.765468   
11   1985-11-02 23:36:09.269886  4.964859        4.936598  14.596208   
12   1985-11-08 12:48:30.101562  5.155676        4.866883   6.313777   
13   1985-11-26 01:15:21.096897  3.529332        4.532495   1.053201   
14   1986-01-11 04:33:36.420703  6.388519        6.388519   3.700551   
15   1986-01-19 17:54:59.206250  4.579486        5.484003   1.921777   
16   1986-02-03 06:12:35.508371  4.946787        5.304931   5.608367   
17   1986-03-01 08:00:17.441840  5.883476        5.415131  17.555518   
18   1986-03-24 09:39:13.101562  5.461074        5.672275  21.441972   
19   1986-04-16 11:18:06.899671  5.962222        5.711648  31.658183   
20   1986-05-09 12:56:55.488838  3.826248        4.894235   1.007251   
21   1986-06-01 14:35:32.742789  5.740774        4.783511  43.136051   
22   1986-06-24 16:14:01.945314  5.193907        5.467341  12.184279   
23   1986-07-17 17:52:39.277575  5.404645        5.299276  10.046763   
24   1986-09-01 21:09:37.889649  5.864396        5.864396  35.232924   
25   1986-09-10 10:30:46.465278  5.581465        5.722930  15.110273   
26   1986-09-24 22:48:05.623699  4.019368        5.155076   2.148464   
27   1986-11-09 14:50:45.132812  4.213162        4.213162   2.035860   
28   1986-11-26 16:03:29.375003  6.019693        5.116427  12.616372   
29   1987-02-02 20:54:26.062498  6.258021        6.258021  22.035394   
...                         ...       ...             ...        ...   
2364 2018-11-21 19:08:25.734747  4.673469        4.759233   2.146051   
2365 2018-11-23 06:07:46.802631  4.665537        4.741180   0.943366   
2366 2018-11-27 07:49:35.633732  4.278931        4.740073   1.744864   
2367 2018-12-01 03:38:59.973047  4.129310        4.788256   0.864849   
2368 2018-12-02 19:27:36.187500  4.718134        4.821506   0.814869   
2369 2018-12-03 04:06:16.772656  5.838010        4.906215   3.941641   
2370 2018-12-07 13:14:13.011719  6.394466        4.956869  74.477695   
2371 2018-12-11 13:20:30.168750  5.328405        4.987830   8.972982   
2372 2018-12-13 02:04:47.866016  4.802386        4.965406   2.638895   
2373 2018-12-16 00:00:11.000000  5.179700        4.988010  19.083675   
2374 2018-12-18 00:09:10.915397  5.459009        5.024241   2.711744   
2375 2018-12-21 18:39:51.439257  4.671923        5.011607   3.971663   
2376 2018-12-23 00:03:22.027146  4.294693        4.980042   1.657978   
2377 2018-12-24 06:26:58.213925  5.879304        5.081189  51.344237   
2378 2018-12-29 18:05:01.030331  5.694865        5.199184  42.698538   
2379 2018-12-31 09:17:33.707032  5.930561        5.349288  37.765601   
2380 2019-01-01 22:01:51.890625  5.495538        5.414072  27.432394   
2381 2019-01-10 06:14:39.021484  4.814379        5.231887   5.478210   
2382 2019-01-11 20:00:23.753125  4.691990        5.174032  10.080063   
2383 2019-01-12 00:00:14.000000  5.088167        5.166876  12.428966   
2384 2019-01-20 05:14:38.116211  4.980003        5.154142  13.863377   
2385 2019-01-21 17:58:56.040625  4.949844        5.181934  10.403953   
2386 2019-01-22 05:01:58.905331  4.982491        5.250714   9.756741   
2387 2019-01-27 16:39:57.302390  5.147659        5.177550  42.26873

In [31]:
print df.date[52]
print df.date[53]

1988-08-01 11:40:58.989583
1992-10-21 07:43:04.942708


In [39]:
#Period normal    
time1 = df.period[0:52]
time2 = df.period[53:-1]

#Period rolling
tt1 = df.period_rolling[0:52]
tt2 = df.period_rolling[53:-1]
        


#Hs normal
y1 = df.wh[0:52]
y2 = df.wh[53:-1]

#Hs rolling
yy1 = df.wh_rolling[0:52]
yy2 = df.wh_rolling[53:-1]


#power normal
p1 = df.power[0:52]
p2=df.power[53:-1]

#power rolling
pp1 = df.power_rolling[0:52]
pp2 = df.power_rolling[53:-1]


#Date
x1=df.date[0:52]
x2 = df.date[53:-1]

In [49]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, sharex = True,figsize = (15,15))    

sns.set_style("white")
#fig.title("Innisfail 30 Day Moving Average Significant Wave Height")


#HS
#Before date break
ax1.plot(x1,y1,color='lightgrey',label="$H_s$")
ax1.plot(x1,yy1,color='blue',label="30-Day Average $H_s$")

#After date break
ax1.plot(x2,y2,color='lightgrey', label='False')
ax1.plot(x2,yy2,color='blue')

ax1.legend(labels=["$H_s$","30-Day Average $H_s$"], loc='upper right')
ax1.set_ylabel("Significant Wave Height (m)")
ax1.set_ylim(0,6)


#Wave Period 
#Before date break
ax2.plot(x1, time1, color='lightgrey')
ax2.plot(x1,tt1,color='blue')

#After date break
ax2.plot(x2,time2, color='lightgrey')
ax2.plot(x2,tt2,color='blue')

ax2.legend(labels=['Period',"30-Day Average Period"], loc='upper right')
ax2.set_ylabel("Wave Period (s)")


#Wave Power - Need to add data
#Before date break
ax3.plot(x1, p1,color='lightgrey')
ax3.plot(x1,pp1,color='blue')

#After date break
ax3.plot(x2,p2,color='lightgrey')
ax3.plot(x2,pp2,color='blue')

ax3.legend(labels=['Period',"30-Day Average Period"], loc='upper right')
ax3.set_ylabel("Wave Energy (kW/m") #Check if watts/metre is correct


#Formatting

plt.xlabel("Year")

#"years = pd.date_range(start='1985',end='2020', periods=10)
years = pd.date_range('2008','2015', freq='AS')
ax1.set_title('Innisfail Timeseries of significant wave height, period and wave power')
plt.savefig("Innisfail_timeseries")

In [34]:
for k in range(len(days)):
    if wh[k]>5:
        print wh[k], alltime[k]

5.654118061065674 1997-03-09 00:27:57.287942


In [35]:
for k in range(len(days)):
    if T[k]>7:
        print wh[k], alltime[k], T[k]

2.1770000457763667 1985-03-31 08:13:46.832721 7.218421169518095
4.651167392730713 1988-06-25 20:13:10.406247 7.55438930172712
1.054800033569336 1993-02-27 05:23:53.878907 7.251031320400107
5.654118061065674 1997-03-09 00:27:57.287942 7.947794289964777
3.821631908416748 1998-01-17 00:58:17.539165 7.188196055037728
1.1669999361038208 1998-10-31 14:15:35.536456 7.471015484727576
1.2774287462234497 1999-02-07 18:00:47.828125 7.115958779782
1.308000087738037 2001-01-30 01:28:51.349608 7.697298430607047
1.036428689956665 2001-07-07 17:05:04.417410 7.266594494628591
0.9443333745002748 2001-08-17 21:43:29.512151 7.071830458390569
0.8014000058174133 2001-10-04 22:51:43.678125 7.067503983548424
1.1776667833328247 2003-01-31 00:00:48.224201 7.242404809755702
3.6381582962839225 2009-01-13 08:17:04.857730 7.975708668017844
1.0576923076923077 2009-11-18 06:15:41.850413 7.295697236394595
1.2784287588936942 2010-03-07 07:59:33.872656 7.424756517792561
0.9838000615437824 2010-11-30 01:19:39.604837 7.03

## Seasonal Kendall Test

In [50]:
r = netCDF4.num2date(tt[days],u'days since 1985-01-01 00:00:00 UTC')

In [78]:
# use this to make year/month df

def getMeanH(month,year):
    valwh = []
    for k in range(len(days)):
        if r[k].year == year and r[k].month == month:
            valwh.append(wh[days[k]])

    return np.mean(valwh)


monthYear = []
for k in range(1,13):
    monthYear.append(getMeanH(k,2018))

print monthYear,
#     print alltime[k].year #,wh[days[k]]

[1.5542418268314449, 1.7217983282398392, 1.5468688456463842, 1.5310160962859816, 1.832533262756629, 1.6641820354911563, 1.1086525499793045, 1.1042811641816683, 1.3621249641233477, 0.8599115383400159, 1.147203009908888, 1.517231036341157]


In [84]:
nameCSV = 'inniswh_my_no_nan_long.csv'
no_nan = pd.read_csv(str(nameCSV))
no_nan.set_index('Date')

Wh
Date            
19931   1.246717
19932   1.656081
19933   0.991064
19934   2.176583
19935   1.996667
19936   1.974246
19937   1.238238
19938   1.567897
19939   1.198661
199310  1.125377
199311  1.052655
199312  1.660867
19941   1.329742
19942   1.435726
19943   1.906962
19944   2.200620
19945   1.595294
19946   1.764080
19947   2.051585
19948   1.899237
19949   1.215860
199410  1.374053
199411  0.757184
199412  1.153817
19951   1.225581
19952   0.751393
19953   1.354757
19954   1.157751
19955   1.468855
19956   1.653992
...          ...
20167   1.324987
20168   1.348094
20169   0.973039
201610  1.139370
201611  1.165785
201612  0.846920
20171   0.794504
20172   1.346251
20173   0.976624
20174   1.678052
20175   1.643524
20176   1.619100
20177   1.526604
20178   1.072012
20179   1.109136
201710  1.076741
201711  1.305514
201712  0.798848
20181   1.554242
20182   1.721798
20183   1.546869
20184   1.531016
20185   1.832533
20186   1.664182
20187   1.108653
20188   1.104281
20189   1.362125
201810  0.859912
201811  1.147203
201812  1.517231

[312 rows x 1 columns]

In [80]:
import pymannkendall as mk
seasonal = mk.seasonal_test(no_nan, period=12)
seasonal

Seasonal_Mann_Kendall_Test(trend='increasing', h=True, p=0.0, z=10.380844483565529, Tau=0.27860911864127297, s=27034.0, var_s=6781458.666666667, slope=0.006047082044117651)

In [83]:
max(df.power)

249.1390066504754

In [48]:
y = no_nan['wh']
max(y)

5.654118061065674